In [1]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward_propagation(self, input):
        raise NotImplementedError
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [2]:
import numpy as np

class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [3]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [4]:
import numpy as np

# Fonction d'activation et sa dérivée 
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

In [5]:
import numpy as np

# Fonction Loss et son dirivée
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [6]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []
        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result
    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                    
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

# MNIST


In [7]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
# normalisation sur train 
x_train /= 255
y_train = np_utils.to_categorical(y_train)
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
# normalisation sur test
x_test /= 255
y_test = np_utils.to_categorical(y_test)

In [8]:
# Network
net = Network()
net.add(FCLayer(28*28, 100))               
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    
net.add(ActivationLayer(tanh, tanh_prime))

net.use(mse, mse_prime)
net.fit(x_train[0:1000], y_train[0:1000], epochs=35, learning_rate=0.1)

epoch 1/35   error=0.206189
epoch 2/35   error=0.103520
epoch 3/35   error=0.082400
epoch 4/35   error=0.067609
epoch 5/35   error=0.055603
epoch 6/35   error=0.047035
epoch 7/35   error=0.040554
epoch 8/35   error=0.035613
epoch 9/35   error=0.031667
epoch 10/35   error=0.028734
epoch 11/35   error=0.026558
epoch 12/35   error=0.024684
epoch 13/35   error=0.022990
epoch 14/35   error=0.021568
epoch 15/35   error=0.020201
epoch 16/35   error=0.018853
epoch 17/35   error=0.017635
epoch 18/35   error=0.016577
epoch 19/35   error=0.015740
epoch 20/35   error=0.014981
epoch 21/35   error=0.014244
epoch 22/35   error=0.013661
epoch 23/35   error=0.013142
epoch 24/35   error=0.012777
epoch 25/35   error=0.012338
epoch 26/35   error=0.011988
epoch 27/35   error=0.011612
epoch 28/35   error=0.011210
epoch 29/35   error=0.010759
epoch 30/35   error=0.010380
epoch 31/35   error=0.010025
epoch 32/35   error=0.009778
epoch 33/35   error=0.009419
epoch 34/35   error=0.009125
epoch 35/35   error=0.0

In [10]:
# Test 
out = net.predict(x_test[0:3])
print("\n")
print("Valeurs prédites : ")
print(out, end="\n")
print("Vraies valeurs : ")
print(y_test[0:3])



Valeurs prédites : 
[array([[-0.01274031,  0.00523753, -0.01291142, -0.01277211, -0.0142871 ,
         0.00355561,  0.00378372,  0.98146792,  0.00663766, -0.04577468]]), array([[ 0.01298352,  0.02474796,  0.30011212,  0.09205669,  0.02696886,
         0.14999438,  0.0543261 ,  0.01820629,  0.39711735, -0.04191284]]), array([[ 0.01354839,  0.9855227 , -0.03048794,  0.26408379,  0.00265519,
         0.00673561,  0.0507668 ,  0.01800227,  0.11860819, -0.07191953]])]
Vraies valeurs : 
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
